# Find Optimal PSF
===========================

- creation 29/06/2016
- author Sylvie Dagoret-Campagne



Find HD158485 with grating

In [13]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from astropy.modeling import models
from astropy import units as u
from astropy import nddata
from astropy.io import fits

from astropy.table import Table
from astropy.table import Column

import ccdproc
print 'ccdproc version',ccdproc.__version__

from astropy.modeling import models

ccdproc version 1.0.1


In [14]:
import photutils
from astropy.stats import sigma_clipped_stats
from photutils import daofind
from photutils import CircularAperture
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize

In [15]:
from scipy import stats 
from scipy import ndimage
import os
from datetime import datetime, timedelta

In [16]:
from photutils.background import Background2D

In [17]:
import libMonocamBaseImages           # my tool library written to do that CCD reduction

In [18]:
now=datetime.utcnow()  # choose UTC time
datestr=str(now)
print 'standard date format for the analysis :',datestr
#  want the following format '2016-05-10T11:55:27.267'
date_of_analysis=now.strftime('%Y-%m-%dT%H:%M:%S')
print 'fits date format for the analysis : ',date_of_analysis

standard date format for the analysis : 2016-06-29 21:29:03.867523
fits date format for the analysis :  2016-06-29T21:29:03


## Definitions and Constants

In [19]:
object_name='HD158485_grat'

### input files

In [20]:
path='./HD158485_grat'
rootfilename='AssScImHD158485_grat_' 
#NumStart=1
#NumStop=50

#NumStart=51
#NumStop=100

NumStart=103
NumStop=132

### output file (table)

In [21]:
outputtablefile='HD158485_grat_103-132_TablePSF.fits'

### make the filelist

In [22]:
filelist=libMonocamBaseImages.BuildFilelist(path,rootfilename,start=NumStart,stop=NumStop,nbchar=1)

In [23]:
filelist

['./HD158485_grat/AssScImHD158485_grat_103.fits',
 './HD158485_grat/AssScImHD158485_grat_104.fits',
 './HD158485_grat/AssScImHD158485_grat_105.fits',
 './HD158485_grat/AssScImHD158485_grat_106.fits',
 './HD158485_grat/AssScImHD158485_grat_107.fits',
 './HD158485_grat/AssScImHD158485_grat_108.fits',
 './HD158485_grat/AssScImHD158485_grat_109.fits',
 './HD158485_grat/AssScImHD158485_grat_110.fits',
 './HD158485_grat/AssScImHD158485_grat_111.fits',
 './HD158485_grat/AssScImHD158485_grat_112.fits',
 './HD158485_grat/AssScImHD158485_grat_113.fits',
 './HD158485_grat/AssScImHD158485_grat_114.fits',
 './HD158485_grat/AssScImHD158485_grat_115.fits',
 './HD158485_grat/AssScImHD158485_grat_116.fits',
 './HD158485_grat/AssScImHD158485_grat_117.fits',
 './HD158485_grat/AssScImHD158485_grat_118.fits',
 './HD158485_grat/AssScImHD158485_grat_119.fits',
 './HD158485_grat/AssScImHD158485_grat_120.fits',
 './HD158485_grat/AssScImHD158485_grat_121.fits',
 './HD158485_grat/AssScImHD158485_grat_122.fits',


## Read Input files

In [24]:
allchannelallsciimages = []  # list of 16 lists of images series 
exposures_list = []        # sequential list of the exposures of the sky flats 
header_list = []           # list of headers
data_list = []
time_list = []                # date and time
basefile_list = []         # basefilename
dateobs_list = [] 
# get the primary block headers:
for image_file in filelist: 
    print image_file
    hdu_list = fits.open(image_file)
    basefile_list.append(os.path.basename(image_file))
    header=hdu_list[0].header
    exposure=header['EXPOSURE']
    exposures_list.append(exposure)
    dateobs_list.append(header['DATE-OBS'])
    header_list.append(header)
    data=ccdproc.CCDData.read(image_file, hdu=0,unit='adu') 
    data_list.append(data)

./HD158485_grat/AssScImHD158485_grat_103.fits
./HD158485_grat/AssScImHD158485_grat_104.fits
./HD158485_grat/AssScImHD158485_grat_105.fits
./HD158485_grat/AssScImHD158485_grat_106.fits
./HD158485_grat/AssScImHD158485_grat_107.fits
./HD158485_grat/AssScImHD158485_grat_108.fits
./HD158485_grat/AssScImHD158485_grat_109.fits
./HD158485_grat/AssScImHD158485_grat_110.fits
./HD158485_grat/AssScImHD158485_grat_111.fits
./HD158485_grat/AssScImHD158485_grat_112.fits
./HD158485_grat/AssScImHD158485_grat_113.fits
./HD158485_grat/AssScImHD158485_grat_114.fits
./HD158485_grat/AssScImHD158485_grat_115.fits
./HD158485_grat/AssScImHD158485_grat_116.fits
./HD158485_grat/AssScImHD158485_grat_117.fits
./HD158485_grat/AssScImHD158485_grat_118.fits
./HD158485_grat/AssScImHD158485_grat_119.fits
./HD158485_grat/AssScImHD158485_grat_120.fits
./HD158485_grat/AssScImHD158485_grat_121.fits
./HD158485_grat/AssScImHD158485_grat_122.fits
./HD158485_grat/AssScImHD158485_grat_123.fits
./HD158485_grat/AssScImHD158485_gr

In [25]:
#basefile_list

## For control

uncomment for control

In [26]:
index=0

In [27]:
#print exposures_list[index]

In [28]:
#header_list[index]

In [29]:
#plt.imshow(data_list[index])

In [30]:
#bkg= Background2D(data_list[index], (100, 100), filter_size=(3, 3),method='median')

In [31]:
#data_list[index].data-bkg.background

## Background subtraction

In [32]:
correctedimage_list = []

In [33]:
for data in data_list:
    bkg= Background2D(data, (100, 100), filter_size=(3, 3),method='median')
    newimage=data-bkg.background
    correctedimage_list.append(newimage)

## Calculation of PSF

In [34]:
NBIMAGES=len(correctedimage_list)

In [35]:
def weighted_avg_and_std(values, weights):
    """
    Return the weighted average and standard deviation.

    values, weights -- Numpy ndarrays with the same shape.
    """
    average = np.average(values, weights=weights)
    variance = np.average((values-average)**2, weights=weights)  # Fast and numerically precise
    return (average, np.sqrt(variance))

In [36]:
image_psf=np.zeros((NBIMAGES,6))

In [37]:
DELTA_NBINSX=10
DELTA_NBINSY=10

In [38]:
# loop on images
index=-1
for image in correctedimage_list:
    index=index+1
    mean, median, std = sigma_clipped_stats(image, sigma=10.0, iters=5) 
    sources = daofind(image - median, fwhm=3.0, threshold=50.*std)
    selected_stars=sources.as_array()
    NBSTARS=selected_stars.shape[0]
    print 'image {} ==> NBSTARS = {}'.format(index,NBSTARS)
    image_psf[index,0]=index
    image_psf[index,1]=NBSTARS
    star_psfx=np.zeros(NBSTARS)
    star_psfy=np.zeros(NBSTARS)
    # loop on stars
    for istar in range(NBSTARS):
        X = int(selected_stars[istar][1])
        Y = int(selected_stars[istar][2])
        prf_image = image[Y-DELTA_NBINSY:Y+DELTA_NBINSY,X-DELTA_NBINSX:X+DELTA_NBINSX]
        oneprfX=prf_image.sum(axis=0)
        oneprfY=prf_image.sum(axis=1)
        if oneprfX.sum() == 0 or oneprfY.sum() == 0:
            star_psfx[istar]=0
            star_psfy[istar]=0
        else:
            posX,sigX=weighted_avg_and_std(np.arange(oneprfX.shape[0]),oneprfX)
            posY,sigY=weighted_avg_and_std(np.arange(oneprfY.shape[0]),oneprfY)
            star_psfx[istar]=sigX
            star_psfy[istar]=sigY
        
    all_sigx=star_psfx[np.logical_not(np.isnan(star_psfx))]
    all_sigy=star_psfy[np.logical_not(np.isnan(star_psfy))]
    all_sigx=all_sigx[all_sigx>2.4]
    all_sigy=all_sigy[all_sigy>2.4]
    print 'average prf(x) = {:2.2f} +/- {:2.2f} pixels ==> psf  {:2.2f} +/- {:2.2f} arcsec '.format(np.median(all_sigx),all_sigx.std(),np.median(all_sigx)*0.4*2.36,all_sigx.std()*0.4*2.26)
    print 'average prf(y) = {:2.2f} +/- {:2.2f} pixels ==> psf  {:2.2f} +/- {:2.2f} arcsec '.format(np.median(all_sigy),all_sigy.std(),np.median(all_sigy)*0.4*2.36,all_sigy.std()*0.4*2.26)
    image_psf[index,2]=np.median(all_sigx)
    image_psf[index,3]=np.median(all_sigy)
    image_psf[index,4]=all_sigx.std()
    image_psf[index,5]=all_sigy.std()

image 0 ==> NBSTARS = 47
average prf(x) = 3.32 +/- 0.24 pixels ==> psf  3.13 +/- 0.21 arcsec 
average prf(y) = 3.10 +/- 1.05 pixels ==> psf  2.92 +/- 0.95 arcsec 


/Users/dagoret-campagnesylvie/anaconda/envs/pyastrophys/lib/python2.7/site-packages/ipykernel/__main__.py:9: RuntimeWarning: invalid value encountered in sqrt


image 1 ==> NBSTARS = 44
average prf(x) = 3.11 +/- 0.21 pixels ==> psf  2.94 +/- 0.19 arcsec 
average prf(y) = 2.98 +/- 0.83 pixels ==> psf  2.81 +/- 0.75 arcsec 
image 2 ==> NBSTARS = 47
average prf(x) = 3.09 +/- 0.25 pixels ==> psf  2.92 +/- 0.23 arcsec 
average prf(y) = 3.29 +/- 0.86 pixels ==> psf  3.11 +/- 0.78 arcsec 
image 3 ==> NBSTARS = 50
average prf(x) = 3.12 +/- 0.25 pixels ==> psf  2.94 +/- 0.22 arcsec 
average prf(y) = 3.03 +/- 1.07 pixels ==> psf  2.86 +/- 0.96 arcsec 
image 4 ==> NBSTARS = 48
average prf(x) = 3.08 +/- 0.24 pixels ==> psf  2.91 +/- 0.22 arcsec 
average prf(y) = 3.07 +/- 0.85 pixels ==> psf  2.90 +/- 0.77 arcsec 
image 5 ==> NBSTARS = 47
average prf(x) = 3.12 +/- 0.36 pixels ==> psf  2.95 +/- 0.32 arcsec 
average prf(y) = 3.38 +/- 0.89 pixels ==> psf  3.19 +/- 0.81 arcsec 
image 6 ==> NBSTARS = 48
average prf(x) = 3.19 +/- 0.26 pixels ==> psf  3.01 +/- 0.24 arcsec 
average prf(y) = 3.22 +/- 0.94 pixels ==> psf  3.04 +/- 0.85 arcsec 
image 7 ==> NBSTARS = 

## Create astropy table

In [39]:
t=Table(rows=image_psf,names=('num','nbstars','prfx','pfry','sig_prfx','sig_prfy'),dtype=('i4','i4','f8','f8','f8','f8'))

In [40]:
t

num,nbstars,prfx,pfry,sig_prfx,sig_prfy
int32,int32,float64,float64,float64,float64
0,47,3.32038393231,3.09662370496,0.236071675366,1.04559112388
1,44,3.11492113162,2.97818295051,0.210843443892,0.826591251253
2,47,3.08815738194,3.29117824128,0.250619993966,0.864441455644
3,50,3.11733721215,3.02954208556,0.246160221458,1.06635290639
4,48,3.08029633915,3.074351356,0.24153928891,0.854841558395
5,47,3.12390314318,3.38156262392,0.356745593817,0.890953896476
6,48,3.19058323033,3.22420936219,0.263373360377,0.93972302274
7,47,3.22451637418,3.12486623724,0.227634535634,0.988243939062
8,53,3.26138545716,3.26057175845,0.228077117386,0.851267523263


In [41]:
expo = Column(exposures_list, name='exposure')
file = Column(basefile_list, name='file')
time = Column(dateobs_list,name='time')

In [42]:
t.add_column(expo, index=1)
t.add_column(time, index=1)
t.add_column(file, index=1)

In [43]:
t

num,file,time,exposure,nbstars,prfx,pfry,sig_prfx,sig_prfy
int32,str29,str23,float64,int32,float64,float64,float64,float64
0,AssScImHD158485_grat_103.fits,2016-05-12T10:29:12.303,5.0,47,3.32038393231,3.09662370496,0.236071675366,1.04559112388
1,AssScImHD158485_grat_104.fits,2016-05-12T10:29:21.641,5.0,44,3.11492113162,2.97818295051,0.210843443892,0.826591251253
2,AssScImHD158485_grat_105.fits,2016-05-12T10:29:30.676,5.0,47,3.08815738194,3.29117824128,0.250619993966,0.864441455644
3,AssScImHD158485_grat_106.fits,2016-05-12T10:29:39.729,5.0,50,3.11733721215,3.02954208556,0.246160221458,1.06635290639
4,AssScImHD158485_grat_107.fits,2016-05-12T10:29:48.819,5.0,48,3.08029633915,3.074351356,0.24153928891,0.854841558395
5,AssScImHD158485_grat_108.fits,2016-05-12T10:29:57.821,5.0,47,3.12390314318,3.38156262392,0.356745593817,0.890953896476
6,AssScImHD158485_grat_109.fits,2016-05-12T10:30:06.868,5.0,48,3.19058323033,3.22420936219,0.263373360377,0.93972302274
7,AssScImHD158485_grat_110.fits,2016-05-12T10:30:15.970,5.0,47,3.22451637418,3.12486623724,0.227634535634,0.988243939062
8,AssScImHD158485_grat_111.fits,2016-05-12T10:30:24.969,5.0,53,3.26138545716,3.26057175845,0.228077117386,0.851267523263


## Write output file

In [44]:
t.write(outputtablefile,format='fits')

## Pandas

In [45]:
df=t.to_pandas()

In [46]:
df.describe()

,num,exposure,nbstars,prfx,pfry,sig_prfx,sig_prfy
count,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000
mean,14.500000,2.133333,23.600000,3.293882,3.241749,0.317081,0.454643
std,8.803408,2.076691,17.621695,0.333514,0.151522,0.268937,0.357247
min,0.000000,0.400000,4.000000,2.982135,2.978183,0.106278,0.070986
25%,7.250000,0.400000,10.000000,3.115525,3.175703,0.200440,0.136120
50%,14.500000,1.000000,15.500000,3.243751,3.219538,0.227856,0.291260
75%,21.750000,5.000000,46.250000,3.348283,3.290000,0.268977,0.853948
max,29.000000,5.000000,53.000000,4.825302,3.686235,1.307125,1.066353
